In [1]:
import pandas as pd
import json
import time
import os
import dill
from docembedder.bert import BERTEmbedder
from docembedder.analysis import DOCSimilarity
from docembedder.preparation import DOCPreparation

In [5]:
# Read and preprocess Patent's documents
patent_document = DOCPreparation()
patent_df = patent_document.read_patents()

# Do some preprocessing here

In [6]:
# Make doc vectors using BERT
patent_embedder = BERTEmbedder()

# Fit the model and save the document embeddings
# patent_embedder.fit(patent_df)

# # Load saved doc vectors- remove this block later
# path = os.path.abspath('models/document_embeddings.model')
# with open(path, 'rb') as f:
#     document_embeddings = dill.load(f)
    
# create look-up-table
patant_analyser = DOCSimilarity()
look_up_table = patant_analyser.create_similarity_matrix()



In [7]:
measure = 'Cosine Similarity'
most_similar_documents, most_similar_document_index = patant_analyser.most_similar(patent_df, 0, look_up_table, measure)
print('\n')
print(f'Document: {most_similar_documents}')
print(f'{measure} : {most_similar_document_index}')

Document: /5 Az izz . A 2zzZe /Zee/ A 2zzezzez ?ez. 6. /853. UNITED STATES PATENT OFFICE. BENJAMIN IRVING, OF GREEN POINT, NEW YORK. IMPROVED PADDLE-WHEEL. Specification forming part of Letters Patent No. 10,000, dated September 6, 1853. - To all uhon itt may concertv: the rhombic form, inasmuch as the side an . Be it known that I, BENJAMIN IRVING, of gles are cut off, and the figure is made six Green Point, in the county of Kings and State sided, as shown in Fig. 4. The best material of New York, have invented a new and use for the floats is sheet-iron, and the several ful Improvement in Paddle-Wheels for Pro floats may be united together and to the pelling Vessels; and I do hereby declare that rings C C by bolts or rivets. The number of the following is a full, clear, and exact de buckets will depend upon the width of the scription of the same, reference being had to wheel. The width of the wheel shown in the accompanying drawings, forming part of cludes three buckets, which I consi